# markdown.obsidian.vault

> Functions for inspecting and modifying Obsidian.md vaults and their folders 

In [ ]:
# TODO: explain what this module is supposed to do.
# TODO: talk about how vault notes are md files.
# TODO: talk about Wikilinks vs Obsidian links

In [ ]:
#| default_exp markdown.obsidian.vault

In [ ]:
#| export
from pathlib import Path
import os
from os import PathLike
from trouver.helper import (
    path_name_no_ext, path_no_ext
)
from typing import Union

In [ ]:
import tempfile
from fastcore.test import *
from nbdev.showdoc import show_doc

## Errors

In [ ]:
#| export
class NoteNotUniqueError(FileNotFoundError):
    """
    A `NoteNotUniqueError` is raised when a `VaultNote` is specified
    by name and not by relative path in the vault, but
    the vault is found to have multiple notes of the name.

    **Attributes**

    - `note_name` - str
        - The name of the note which is not unique in the vault.
    - `notes` - list[str]
        - The paths of the notes whose names are `note_name`.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)
            
    @classmethod
    def from_note_names(cls, note_name: str, notes: list[str]):
        """Construct a `NoteNotUniqueError` object from note names"""
        return cls(
            f'The note of the following name is not unique: {note_name}\n'\
            f'The name points to the following files: {notes}')



In [ ]:
show_doc(NoteNotUniqueError.from_note_names)

---

### NoteNotUniqueError.from_note_names

>      NoteNotUniqueError.from_note_names (note_name:str, notes:list[str])

Construct a `NoteNotUniqueError` object from note names

In [ ]:
#| export
class NoteDoesNotExistError(FileNotFoundError):
    """
    A `NoteDoesNotExistError` is raised when a `VaultNote` is specified
    by either name and or by relative path in the vault, but
    the vault is found to have no notes of the name.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_note_name(cls, note_name: str):
        """Construct a `NoteDoesNotExistError` object from note name"""
        return cls(
            f'The note of the following name does not exist: {note_name}.'
            f' Make sure that the argument passed to `note_name` does not'
            f' erroneously end with `.md`, e.g. pass `this_is_a_note`'
            f' instead of `this_is_a_note.md`.')

## Convert path to Obsidian ID

In [ ]:
#| export
def path_to_obs_id(
        rel_path: PathLike # A path representation the path of an Obsidian note relative to its vault. This does not have to be an existing path.
        ) -> str: # The obsidian url of the hypothetical note within its vault.
    """Convert a relative path of an Obsidian note to the Obsidian identifying
    str.
    
    This identification is for a vault-internal Wikilink.

    Note that this function does not have a vault as a parameter.
    """
    path_without_extension = path_no_ext(rel_path)
    return path_without_extension.replace('\\', '/')

Obsidian formats its file paths with '/'; the `path_to_obs_id` function converts a relative path of an Obsidian note to the Obsidian-recognized path.

In [ ]:
assert path_to_obs_id(
    r'algebraic_geometry\foag\III_morphisms\8_closed_embeddings_and_related_notions\1_closed_embeddings_and_closed_subschemes\sheaf_of_ideals_on_a_scheme.md')\
    == 'algebraic_geometry/foag/III_morphisms/8_closed_embeddings_and_related_notions/1_closed_embeddings_and_closed_subschemes/sheaf_of_ideals_on_a_scheme'

Obsidian notes might contain spaces in their paths.

In [ ]:
assert path_to_obs_id(
    Path('number_theory/18785/9_local_fields_and_hensels_lemmas/1_local_fields/18785_Theorem 9.9.md')
) == 'number_theory/18785/9_local_fields_and_hensels_lemmas/1_local_fields/18785_Theorem 9.9' 

## Get all notes

In this subsection, we take an example vault with the following files and folders:

```
root
+-- algebra
    +-- altman_kleiman_ca
        +-- ring.md
    +-- ring.md
+-- analysis
+-- category_theory
    +-- category.md
+-- _index.md
```

Note that it has $4$ files and $4$ folders in total.

In [ ]:
def make_example_vault(temp_dir: PathLike):
    temp_dir = Path(temp_dir)
    os.mkdir(temp_dir / 'algebra')
    os.mkdir(temp_dir / 'algebra' / 'altman_kleiman_ca')
    os.mkdir(temp_dir / 'analysis')
    os.mkdir(temp_dir / 'category_theory')
    (temp_dir / 'algebra' / 'ring.md').touch()
    (temp_dir / 'algebra' / 'altman_kleiman_ca' / 'ring.md').touch()
    (temp_dir / 'category_theory' / 'category.md').touch()
    (temp_dir / '_index.md').touch()

In [ ]:
#| export
def all_paths_to_notes_in_vault(
        vault: PathLike, as_dict: bool = False)\
        -> Union[list[str], dict[str, list[str]]]:
    """Returns the paths, relative to the Obsidian vault, of notes 
    in the Obsidian vault.
       
    This may not actually return all of the paths to the notes, see
    the parameter `as_dict`.

    **Parameters**

    - `vault` - `PathLike`
        - The path to the Obsidian vault directory
    - `as_dict` - `bool`
        - If `True`, then returns a dict. If `False`, then returns
        a list. Defaults to `False`. If there are multiple notes with the same
        name in the vault, and `as_dict` is set to `True`, then the dictionary
        will contain only one of the (relative) paths to those notes among its
        values. If `as_dict` is set to `False`, then the list will contain
        all the paths to the notes even when notes with non-unique names exist.
        
    **Returns**

    - Union[list[str], dict[str, str]]
        - Each str represents the path relative to `vault`. If `as_dict` is
        True, then returns a dict whose keys are str, which are (unique) names
        of the notes in the vault, and the values are the paths.
    """
    vault = Path(vault)
    paths =  [os.path.relpath(path, vault) for path in vault.glob(f'**/*.md')]
    if as_dict:
        dicty = {path_name_no_ext(path): [] for path in paths}
        for path in paths:
            dicty[path_name_no_ext(path)].append(path)
        return dicty
    else:
        return paths


The `all_paths_to_notes_in_vault` function returns all of the paths to notes in the Obsidian vault; only `.md` files are recognized as notes.

By default, the function returns a list whose items are strings of paths to notes relative to the vault path.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    make_example_vault(temp_dir)
    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_dir)
    test_eq(len(notes), 4)
    print(notes)
    # test_shuffled(, list)
    

['_index.md', 'algebra\\ring.md', 'algebra\\altman_kleiman_ca\\ring.md', 'category_theory\\category.md']


Passing `as_dict=True` returns a dictionary whose keys are note names and whose values are lists of paths to notes of the name relative to the vault.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    make_example_vault(temp_dir)
    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_dir, as_dict=True)
    test_eq(len(notes), 3)
    test_eq(len(notes['ring']), 2)
    print(notes)


{'_index': ['_index.md'], 'ring': ['algebra\\ring.md', 'algebra\\altman_kleiman_ca\\ring.md'], 'category': ['category_theory\\category.md']}


## Searching notes by name

In [ ]:
#| export
def all_note_paths_by_name(
        name: str, vault: PathLike,
        subdirectory: Union[PathLike, None] = None) -> list[Path]:
    """Returns the relative paths to all notes in the Obsidian vault 
    with the specified name in the specified subdirectory.
    
    This function does not assume that the specified subdirectory in the vault
    has at most one note of the specified name.
    
    **Parameters**
    - `name` - str
    - `vault` - PathLike
        - The path to the Obsidian vault directory
    - `subdirectory` - Union[PathLike, None]
        - The path to a subdirectory in the Obsidian vault, relative
        to `vault. If `None`, then denotes the root of the vault. 
        Defaults to `None`.
        
    **Returns**
    - list[Path]
        - Each item is a path to a note of the given name, relative to
        `vault`.
    """
    vault = vault if vault != None else ''
    vault = Path(vault)
    subdirectory = subdirectory if subdirectory != None else ''
    directory_path = vault / subdirectory
    all_notes_of_name = directory_path.glob(f'**/{name}.md')
    all_notes_of_name = list(all_notes_of_name)
    return [note_path.relative_to(vault) for note_path in all_notes_of_name]

Basic usage: